<a href="https://colab.research.google.com/github/gpandu/CodeGenGPT/blob/main/CodeGPT_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Training GPT model is done using following steps.
1. Load and prepare dataset for tokenizer training.
2. Train BPE tokenizer from scratch.
3. Prepare tensorflow dataset with generator.
4. Create layer and model classes for GPT



*   We will use tokenizers library from HuggingFace to train tokenizer from scratch.
*   We will also use datasets to load the python "code_search_net" dataset. It has ~410k of training records. If we load the load at once we will run out of RAM, so we will take advantage streaming the batches.




In [2]:
!pip install tokenizers
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.4 MB/s eta 0:00:00


In [3]:
import math
import numpy as np
import tensorflow as tf
from datasets import load_dataset

from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)


*   Load the "code_search_net" adn we can check details of the dataset like below.



In [ ]:


# load code dataset
raw_dataset = load_dataset("code_search_net", "python")
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
        num_rows: 412178
    })
    test: Dataset({
        features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
        num_rows: 22176
    })
    validation: Dataset({
        features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
        num_rows: 23107
    })
})



*   Create a generator to load the data in batches for training tokenizer.

   >  Loading all the data at once may cause out of memory error.







In [ ]:
def get_training_corpus():
    dataset = raw_dataset["train"]
    for start_idx in range(0, len(dataset), 1000):
        samples = dataset[start_idx : start_idx + 1000]
        yield samples["whole_func_string"]


In [ ]:
#check if we are able to iterate over the dataset.
iterator = iter(get_training_corpus())
next(iterator)[0]

'def export_ruptures_csv(ekey, dstore):\n    """\n    :param ekey: export key, i.e. a pair (datastore key, fmt)\n    :param dstore: datastore object\n    """\n    oq = dstore[\'oqparam\']\n    if \'scenario\' in oq.calculation_mode:\n        return []\n    dest = dstore.export_path(\'ruptures.csv\')\n    header = (\'rupid multiplicity mag centroid_lon centroid_lat \'\n              \'centroid_depth trt strike dip rake boundary\').split()\n    rows = []\n    for rgetter in gen_rupture_getters(dstore):\n        rups = rgetter.get_ruptures()\n        rup_data = calc.RuptureData(rgetter.trt, rgetter.rlzs_by_gsim)\n        for r in rup_data.to_array(rups):\n            rows.append(\n                (r[\'rup_id\'], r[\'multiplicity\'], r[\'mag\'],\n                 r[\'lon\'], r[\'lat\'], r[\'depth\'],\n                 rgetter.trt, r[\'strike\'], r[\'dip\'], r[\'rake\'],\n                 r[\'boundary\']))\n    rows.sort()  # by rupture serial\n    comment = \'investigation_time=%s, ses_per

Tokenization:



*   Subword Tokenization : Keep frequent words and break rearer words into subwords
*   A statastical Alogrothm learns how to do this based on corpus.

> Ex: Listeria ---> "more" , "over"

> "more" and "over" are likely to be more frequent than moreover


*   Tokenization has better chance of handling OOV words while decreasing the size of the overall dictionary.   

* We will use BPE(Byte Pair Encoding) to train tokenizer on "code_search_net" python Dataset.
*   For more information on BPE can be found here. https://huggingface.co/learn/nlp-course/chapter6/5?fw=pt


In [ ]:
tokenizer = Tokenizer(models.BPE())
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)

# "<|endoftext|>" will used to stop the sequence generation during inference. This is also
#  a way telling GPT to learn to about the end of the sequence
trainer = trainers.BpeTrainer(vocab_size=25000, special_tokens=["<|endoftext|>"])

#Train the tokenizer using BPE trainer, loads the data in batches
tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)


*    Check if we are able to tokenize and encode the data using trained BPE tokenizer
*   Actual maximum sequence length MAX_SEQ_LENGTH = 256, we will add one to 256 so that last sample will dropped for the inputs and first sample will be dropped from the outputs. This makes model to see only previous samples to predict next sample.

        For example:
                 input  :     295    4354   63    72      6035   63
                 output :     4354   63     72    6035    63     3170




In [ ]:
example = '''def add_numbers(a, b):
    """Add the two numbers `a` and `b`."""
    return a + b'''

tokenizer.enable_padding(length=257, pad_id = 0, pad_token = "<|endoftext|>")
tokenizer.enable_truncation(max_length=257)
encoding = tokenizer.encode(raw_dataset["train"][1]["whole_func_string"])
print(encoding.ids)
print(encoding.attention_mask)

[295, 4354, 63, 72, 6035, 63, 3170, 8, 368, 12, 1761, 12, 2840, 4475, 12, 960, 12, 3184, 274, 232, 290, 232, 14210, 256, 9081, 6778, 311, 256, 868, 2899, 1597, 1202, 8446, 14, 312, 310, 394, 521, 26, 819, 63, 369, 350, 4354, 63, 369, 232, 310, 394, 1761, 26, 455, 311, 256, 10245, 480, 232, 310, 394, 2840, 4475, 26, 2840, 2303, 232, 310, 394, 960, 26, 231, 11512, 960, 311, 1719, 366, 923, 63, 3112, 232, 310, 394, 3184, 26, 3184, 292, 811, 442, 1608, 311, 256, 10245, 8446, 480, 232, 290, 232, 12493, 233, 4705, 14, 11080, 63, 7194, 8, 2833, 4475, 12, 960, 9, 232, 2968, 501, 14, 1025, 63, 3170, 8, 2603, 12, 12493, 12, 3184, 29, 2124, 9, 232, 302, 404, 2603, 61, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0



*   Generator to prepare inputs and outputs in the batches.
*   Inputs and Outputs will have sequences ids encoded from the tokenizer.



In [ ]:
batch_size = 50
def generate_train_data():
  decoder_inputs = []
  decoder_targets = []
  dataset = raw_dataset["train"]
  for start_idx in range(0, len(dataset), batch_size):
      samples = dataset[start_idx : start_idx + batch_size]
      seqs = tokenizer.encode_batch(samples["whole_func_string"])
      decoder_inputs = [seq.ids[:-1] for seq in seqs] # Drop the last token in the sentence.
      decoder_targets = [seq.ids[1:] for seq in seqs]  # Drop the first token in the sentence.
      yield decoder_inputs, decoder_targets


In [ ]:
#tf_dataset = raw_dataset["train"].to_tf_dataset(batch_size = 100, columns = ['whole_func_string'])
#tf_dataset

<_PrefetchDataset element_spec=TensorSpec(shape=(None,), dtype=tf.string, name=None)>

In [ ]:
iterator = iter(generate_train_data())
decoder_inputs, decoder_targets = next(iterator)
print(decoder_inputs[80])


[295, 627, 52, 18, 48, 13069, 8, 1317, 274, 232, 290, 232, 10197, 231, 19536, 238, 7287, 311, 231, 868, 7335, 3995, 14, 312, 310, 394, 11348, 26, 310, 692, 2310, 4535, 407, 5595, 64, 232, 310, 780, 26, 310, 692, 2310, 46, 395, 238, 48, 13069, 64, 312, 20346, 301, 497, 23110, 8516, 3097, 232, 572, 6313, 14, 77, 659, 1651, 1193, 3883, 14, 2936, 73, 14, 240, 299, 72, 320, 14, 11016, 15, 23701, 15, 338, 619, 12012, 15, 15678, 2561, 15, 15678, 2561, 14, 1825, 7564, 232, 4547, 585, 8898, 89, 627, 889, 65, 336, 350, 690, 279, 347, 4512, 12012, 14, 232, 290, 232, 308, 68, 12, 486, 9, 233, 635, 14, 5908, 14, 12657, 8, 1317, 14, 1317, 9, 232, 523, 233, 635, 14, 783, 929, 68, 59, 17, 547, 270, 59, 16, 547, 270, 59, 18, 4211, 232, 641, 233, 635, 14, 783, 5773, 86, 59, 17, 12, 396, 547, 415, 86, 59, 17, 12, 443, 547, 415, 86, 59, 17, 12, 554, 4936, 1797, 404, 86, 59, 18, 12, 396, 547, 415, 86, 59, 18, 12, 443, 547, 415, 86, 59, 18, 12, 554, 4936, 1797, 4866, 86, 59, 16, 12, 396, 547, 486, 59, 16, 1

In [ ]:
#tf_dataset = tf.data.Dataset.from_tensor_slices((decoder_inputs, decoder_targets)).batch(10, drop_remainder=True)

tf_dataset = tf.data.Dataset.from_generator(generate_train_data, output_types=(tf.int32, tf.int32), output_shapes=(tf.TensorShape([50,256]), tf.TensorShape([50,256])))

# checks to see if data is loading properly.
iterator = iter(tf_dataset)
ins, outs = next(iterator)
print(ins.shape)
next(iterator)[0]

<tf.Tensor: shape=(10, 256), dtype=int32, numpy=
array([[ 295,  549,   63, ...,  357, 3665,   19],
       [ 295,  409, 8485, ...,    0,    0,    0],
       [ 295,  409,  322, ...,    0,    0,    0],
       ...,
       [ 295, 1339,    8, ...,    0,    0,    0],
       [ 295, 2274,   63, ..., 3921,   63,   66],
       [ 295,  549,   63, ...,    0,    0,    0]], dtype=int32)>

Multi Head Attention



*   Each Attention head performs Scaled Dot Product Self-Attention operation where given Keys, Query and Values, the return matrix of values given by below operation.

        Attention(Q,K,V) = softmax((Q*Transpose(K))/sqrt(d))*V





In [ ]:
def scaled_dot_product_attention(query, key, value, mask=None):
  key_dims = tf.cast(tf.shape(key)[-1], tf.float32)
  scaled_scores = tf.matmul(query, key, transpose_b=True) / tf.math.sqrt(key_dims)

  if mask is not None:
    scaled_scores = tf.where(mask==0, -np.inf, scaled_scores)

  softmax = tf.keras.layers.Softmax()
  weights = softmax(scaled_scores)
  return tf.matmul(weights, value), weights



**Generating queries, keys, and values for multiple heads.**

> Now that we have a way to calculate self-attention, let's actually generate the input queries, keys, and values for multiple heads.

>  each attention head had its own separate set of query, key, and value weights. Each weight matrix was of dimension  d x d/h  where h was the number of heads.




In [ ]:
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.d_model = d_model
    self.num_heads = num_heads

    self.d_head = self.d_model // self.num_heads

    self.wq = tf.keras.layers.Dense(self.d_model, use_bias=False)
    self.wk = tf.keras.layers.Dense(self.d_model, use_bias=False)
    self.wv = tf.keras.layers.Dense(self.d_model, use_bias=False)

    # Linear layer to generate the final output.
    self.dense = tf.keras.layers.Dense(self.d_model)

  def split_heads(self, x):
    batch_size = x.shape[0]

    split_inputs = tf.reshape(x, (batch_size, -1, self.num_heads, self.d_head))
    return tf.transpose(split_inputs, perm=[0, 2, 1, 3])

  def merge_heads(self, x):
    batch_size = x.shape[0]

    merged_inputs = tf.transpose(x, perm=[0, 2, 1, 3])
    return tf.reshape(merged_inputs, (batch_size, -1, self.d_model))

  def call(self, q, k, v, mask):
    qs = self.wq(q)
    ks = self.wk(k)
    vs = self.wv(v)

    qs = self.split_heads(qs)
    ks = self.split_heads(ks)
    vs = self.split_heads(vs)

    output, attn_weights = scaled_dot_product_attention(qs, ks, vs, mask)
    output = self.merge_heads(output)

    return self.dense(output), attn_weights


Feed Forward Neural Network

In [ ]:
def feed_forward_network(d_model, hidden_dim):
  return tf.keras.Sequential([
      tf.keras.layers.Dense(hidden_dim, activation='relu'),
      tf.keras.layers.Dense(d_model)
  ])

Decode Block

In [ ]:
class DecoderBlock(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, hidden_dim, dropout_rate=0.1):
    super(DecoderBlock, self).__init__()

    self.mhsa1 = MultiHeadAttention(d_model, num_heads)

    self.ffn = feed_forward_network(d_model, hidden_dim)

    self.dropout1 = tf.keras.layers.Dropout(dropout_rate)
    self.dropout2 = tf.keras.layers.Dropout(dropout_rate)

    self.layernorm1 = tf.keras.layers.LayerNormalization()
    self.layernorm2 = tf.keras.layers.LayerNormalization()

  def call(self, target, training, decoder_mask):
    mhsa_output1, attn_weights = self.mhsa1(target, target, target, decoder_mask)
    mhsa_output1 = self.dropout1(mhsa_output1, training=training)
    mhsa_output1 = self.layernorm1(mhsa_output1 + target)

    ffn_output = self.ffn(mhsa_output1)
    ffn_output = self.dropout2(ffn_output, training=training)
    output = self.layernorm2(ffn_output + mhsa_output1)

    return output, attn_weights


Decoder with Mulitple Layers

In [ ]:
class Decoder(tf.keras.layers.Layer):
  def __init__(self, num_blocks, d_model, num_heads, hidden_dim, target_vocab_size,
               max_seq_len, dropout_rate=0.1):
    super(Decoder, self).__init__()

    self.d_model = d_model
    self.max_seq_len = max_seq_len

    self.token_embed = tf.keras.layers.Embedding(target_vocab_size, self.d_model)
    self.pos_embed = tf.keras.layers.Embedding(max_seq_len, self.d_model)

    self.dropout = tf.keras.layers.Dropout(dropout_rate)

    self.blocks = [DecoderBlock(self.d_model, num_heads, hidden_dim, dropout_rate) for _ in range(num_blocks)]

  def call(self, input, training, decoder_mask):
    token_embeds = self.token_embed(target)

    # Generate position indices.
    num_pos = input.shape[0] * self.max_seq_len
    pos_idx = np.resize(np.arange(self.max_seq_len), num_pos)
    pos_idx = np.reshape(pos_idx, input.shape)

    pos_embeds = self.pos_embed(pos_idx)

    x = self.dropout(token_embeds + pos_embeds, training=training)

    for block in self.blocks:
      x, weights = block(x, training, decoder_mask)

    return x, weights

Custom loss function to remove effect of padding

In [ ]:
def loss_func(targets, logits):
  ce_loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  mask = tf.cast(tf.math.not_equal(targets, 0), tf.float32)
  return ce_loss(targets, logits, sample_weight=mask)

In [ ]:
class GPTModel(tf.keras.Model):
  def __init__(self, num_blocks, d_model, num_heads, hidden_dim, target_vocab_size,
               max_input_len, dropout_rate=0.1):
    super(GPT2Model, self).__init__()

    self.decoder = Decoder(num_blocks, d_model, num_heads, hidden_dim, target_vocab_size,
                           max_input_len, dropout_rate)

    # The final dense layer to generate logits from the model output.
    self.output_layer = tf.keras.layers.Dense(target_vocab_size)

  @tf.function
  def train_step(self, inputs):
      loss = 0.

      decoder_input_seq, targets = inputs
      with tf.GradientTape() as tape:

        dec_padding_mask = tf.cast(tf.math.not_equal(decoder_input_seq, 0), tf.float32)
        dec_padding_mask = dec_padding_mask[:, tf.newaxis, tf.newaxis, :]
        target_input_seq_len = len(decoder_input_seq[0])
        look_ahead_mask = tf.linalg.band_part(tf.ones((target_input_seq_len,
                                               target_input_seq_len)), -1, 0)
        dec_mask = tf.minimum(dec_padding_mask, look_ahead_mask)

        logits, _ = self.decoder(decoder_input_seq, True, dec_mask)
        logits =   self.output_layer(logits)
        loss += self.loss(targets, logits)

      # Update the parameters and the optimizer
      variables = self.decoder.trainable_variables
      gradients = tape.gradient(loss, variables)
      self.optimizer.apply_gradients(zip(gradients, variables))

      return {'loss': loss}

  def call(self, target, training):
    logits, _ = self.decoder(target, True, None)
    logits =   self.output_layer(logits)
    return logits



In [ ]:
model = GPTModel(
    num_blocks = 6,
    d_model = 512,
    num_heads = 4,
    hidden_dim = 1024,
    target_vocab_size = tokenizer.get_vocab_size(),
    max_input_len = 256)

optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer, loss=loss_func, run_eagerly=True)

In [ ]:
epochs = 10
model.fit(tf_dataset, epochs=epochs)

Epoch 1/10
    847/Unknown - 582s 668ms/step - loss: 6.1027